In [ ]:
import os
import sys
sys.path.append(os.path.join(os.path.abspath("../.."), "functions"))

import pandas as pd

## Traffic per edge

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib.colors import Normalize
from constants import CRS_PROJECTED

In [ ]:
def plot_traffic(
    traffic_data, 
    hour_plot:int, 
    color_variable:str, 
    title=None, figsize=(10, 7), 
    cmap='viridis_r', 
    min_value=None, 
    max_value=None,
    alpha_map=0.25
):
    
    # Filter data for the specified hour
    hour_data = traffic_data[traffic_data['hour'] == hour_plot]
    hour_data = hour_data.sort_values(by=color_variable)
    
    if min_value is None:
        min_value = hour_data[color_variable].min()
    if max_value is None:
        max_value = hour_data[color_variable].max()
    norm = Normalize(vmin=min_value, vmax=max_value)
    color_map = plt.get_cmap(cmap)
    
    fig, ax = plt.subplots(figsize=figsize)

    hour_data.plot(ax=ax, column=color_variable, cmap=color_map, linewidth=1, legend=True, norm=norm, missing_kwds={'color': 'grey'})
    
    ctx.add_basemap(ax=ax, crs=CRS_PROJECTED, source=ctx.providers.OpenStreetMap.Mapnik, alpha=alpha_map)
    
    if title is None:
        title = f"{color_variable} per road between {hour_plot} and {hour_plot+1}"
    plt.title(title)
    
    return fig, ax

In [ ]:
# Compute traffic
from uxsimulator.analysis import edge_metrics

dp = "results"
th = 24
list_hours = [i for i in range(th)]
ts = 5
list_seeds = [i for i in range(ts)]
nn = "nodes_v7"
nl = "UXsim_links/AreaVerde_links_with_return_v5"

traffic = edge_metrics.calculate_traffic(
    list_hours=list_hours, 
    list_seeds=list_seeds,
    datapath=dp, 
    namefile_nodes=nn, 
    namefile_traffic=nl
)

In [ ]:
# Add link geometry
import geopandas as gpd
from constants import CRS_PROJECTED

edges = gpd.read_file("results/geo_edges_v5.geojson")[['link_id', 'maxspeed_imputed', 'geometry']].rename(columns={'link_id': 'link', 'geometry': 'geometry_link'})
traffic = (
    traffic
    .drop(columns=['seed', 'vehicles_remain'])
    .merge(edges)
    .set_geometry('geometry_link')
    .set_crs(CRS_PROJECTED)
)

In [ ]:
#New columns
# Density
traffic['traffic_density'] = traffic['traffic_volume'] / traffic['length']
print(traffic['traffic_density'].describe())

# Speed
traffic.loc[traffic['average_travel_time'] < 0, 'average_travel_time'] = None
traffic['free_speed'] = traffic['length'] / traffic['free_travel_time']
traffic['average_speed'] = traffic['length'] / traffic['average_travel_time']
traffic['delta_speed'] =  -traffic['average_speed'] + traffic['free_speed']
traffic['ratio_speed'] =  traffic['average_speed'] / traffic['free_speed']

In [ ]:
# Traffic volume
h = 8
fig, ax = plot_traffic(
    traffic_data=traffic,
    hour_plot=h,
    color_variable='traffic_volume',
    title=f"Traffic volume [veh] per road between {h} and {h+1}"
)
plt.show()

In [ ]:
# Traffic density
h = 8
fig, ax = plot_traffic(
    traffic_data=traffic,
    hour_plot=h,
    color_variable='traffic_density',
    title=f"(95%) Traffic density [veh/m] per road between {h} and {h+1}",
    min_value=0,
    max_value=25
)
plt.show()

In [ ]:
# Average time
h = 8
fig, ax = plot_traffic(
    traffic_data=traffic,
    hour_plot=h,
    color_variable='average_travel_time',
    min_value=1,
    alpha_map=0.5,
    title=f"Average travel time [s] per road between {h} and {h+1}"
)
plt.show()

In [ ]:
# Average speed
h = 8
fig, ax = plot_traffic(
    traffic_data=traffic,
    hour_plot=h,
    color_variable='average_speed',
    min_value=1,
    alpha_map=0.5,
    title=f"Average speed [m/s] per road between {h} and {h+1}"
)
plt.show()

In [ ]:
id10 = (traffic['hour']==8)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Plot histogram for delta_speed on the first axis
ax1.hist(traffic.loc[id10, 'delta_speed'], bins=50)
ax1.set_title('Delta Speed Distribution')
ax1.set_xlabel('Delta Speed')
ax1.set_ylabel('Frequency')

# Plot histogram for ratio_speed on the second axis
ax2.hist(traffic.loc[id10, 'ratio_speed'], bins=50)
ax2.set_title('Ratio Speed Distribution')
ax2.set_xlabel('Ratio Speed')
ax2.set_ylabel('Frequency')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
traffic.to_file("results/AreaVerde_T_roads/AreaVerde_T_roads_v14.geojson", driver="GeoJSON")

## IO per zone

In [ ]:
from uxsimulator.analysis import zone_metrics

th = 24
list_hours = [i for i in range(th)]
ts = 5
list_seeds = [i for i in range(ts)]
dp = "results"
ne = "edges_v7"
nn = "nodes_v7"
nv = "UXsim_vehicles/AreaVerde_vehicles_with_return_v5"
nsave = "AreaVerde_IO/AreaVerde_IO_v14"
df_io = zone_metrics.compute_full_table(
    table_type="IO", list_hours=list_hours, list_seeds=list_seeds,
    datapath=dp, namefile_edges=ne, namefile_nodes=nn, namefile_vehicles=nv,
    namefile_save=nsave, verbose=True
)
df_io.head()

## S per zone

In [ ]:
from uxsimulator.analysis import zone_metrics

th = 24
list_hours = [i for i in range(th)]
ts = 5
list_seeds = [i for i in range(ts)]
dp = "results"
ne = "edges_v7"
nv = "UXsim_vehicles/AreaVerde_vehicles_with_return_v5"
nsave = "AreaVerde_S/AreaVerde_S_v14"

df_s = zone_metrics.compute_full_table(
    table_type="S", list_hours=list_hours, list_seeds=list_seeds,
    datapath=dp, namefile_edges=ne, namefile_vehicles=nv,
    namefile_save=nsave, verbose=True
)
df_s.head()

## Traffic per zone

In [ ]:
import uxsimulator.analysis.zone_metrics

th = 24
list_hours = [i for i in range(th)]
ts = 5
list_seeds = [i for i in range(ts)]
dp = "results"
ne = "edges_v7"
nv = "UXsim_vehicles/AreaVerde_vehicles_with_return_v5"
nsave = "AreaVerde_T/AreaVerde_T_v14"

df_t = uxsimulator.analysis.zone_metrics.compute_full_table(
    table_type="T", list_hours=list_hours, list_seeds=list_seeds,
    datapath=dp, namefile_edges=ne, namefile_vehicles=nv,
    namefile_save=nsave, verbose=True
)
df_t.head()


## Check the output variability

In [ ]:
# Check the sign of the derivative
import pandas as pd

typedata = "IO"

df = pd.read_parquet(f"results/AreaVerde_{typedata}/AreaVerde_{typedata}_v14.parquet")

col = "starting_from_zone_mean" if typedata=="S" else ("traffic_in_zone_mean" if typedata=="T" else "inflow_from_INSIDE_mean")
col2 = f"{col}_diff"
col3 = f"{col2}_sign"

df[col2] = df.groupby(['id_zone'])[col].transform(lambda x: x.diff())
df = df[df['hour']!=0]
df[col3] = df.apply(lambda x: 1 if x[col2]>=0 else -1, axis=1)

count_per_hour = df[df[col3] == 1].groupby('hour')['id_zone'].count()
total_zones_per_hour = df.groupby('hour')['id_zone'].count()
percentage_per_hour = (count_per_hour / total_zones_per_hour) * 100
percentage_per_hour = percentage_per_hour.fillna(0)

import matplotlib.pyplot as plt

percentage_per_hour.plot(kind='bar', figsize=(8,4))

plt.title('Percentage of Zones with +1 in Sign per Hour')
plt.xlabel('Hour')
plt.ylabel('Percentage')
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

ts = 5
list_seeds = range(ts)
th = 24
list_hours = range(th)
zoneid = 91

for typedata in ["T", "S", "IO"]:
    col1 = "starting_from_zone" if typedata=="S" else ("traffic_in_zone" if typedata=="T" else "inflow_from_INSIDE")
    col2 = "inflow_from_OUTSIDE" if typedata=="IO" else None

    df_all2 = pd.read_parquet(f"results/AreaVerde_{typedata}/AreaVerde_{typedata}_v14.parquet")
    df_all2 = df_all2[df_all2['id_zone']==zoneid]

    for seed in list_seeds:
        for hour in list_hours:
            tmp = pd.read_parquet(f"results/AreaVerde_{typedata}/AreaVerde_{typedata}_v14_from_{hour}_to_{hour+1}_seed_{seed}.parquet")
            tmp = tmp[tmp['id_zone']==zoneid]
            tmp['hour'] = hour
            tmp['seed'] = seed
            df_filtered = tmp if ((seed==list_seeds[0]) & (hour==list_hours[0])) else pd.concat([tmp, df_filtered])

    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    ax.plot(df_all2['hour'], df_all2[f"{col1}_mean"], color='navy', label=col1)
    if col2 is not None:
        ax.plot(df_all2['hour'], df_all2[f"{col2}_mean"], color='red', label=col2)

    plt.legend()

    for seed, group in df_filtered.groupby('seed'):
        ax.plot(group['hour'], group[col1], linestyle='--', alpha=0.4, linewidth=1, color='blue',label=None) 
        if col2 is not None:
            ax.plot(group['hour'], group[col2], linestyle='--', alpha=0.7, linewidth=1, color='lightcoral',label=None)
        
    plt.title(f"Zone {zoneid}")
    plt.grid()
    plt.show()

## Travel times

In [ ]:
import uxsimulator.analysis.utils
import ast

th = 24
list_h = [i for i in range(3,24)] + [i for i in range(3)]
ts = 1
list_s = [i for i in range(ts)]
namefile = "UXsim_vehicles/AreaVerde_vehicles_with_return_v5"
dp = "results"

for s in list_s:
    print(s)

    travel_times_not_finished = {}
    travel_times_finished = {}
    for h in list_h:
        list_prev_h = list_h[:list_h.index(h)][::-1]
        print(f"Siamo nell'ora {h}------------------------------------------------------")
        
        travel_times_not_finished[h] = {}
        travel_times_finished[h] = {}

        veh = uxsimulator.analysis.utils.read_output(namefile=f"{namefile}_from_{h}_to_{h+1}_seed_{s}.csv", datapath=dp, dtype={0:str})
        
        veh['attribute'] = veh['attribute'].apply(ast.literal_eval)
        veh_2 = veh['attribute'].apply(pd.Series)
        veh = veh.join(veh_2)

        grouped = veh.groupby('name')

        for name, group in grouped:
            is_new = False
            is_completed = False

            group_sorted = group.sort_values('t')

            min_time = group_sorted['t'].iloc[0]
            max_time = group_sorted['t'].iloc[-1]

            if any(group_sorted['prev_name'] != ''):
                min_all = 0
            else:
                is_new = True
                min_all = min_time

            if any(group_sorted['link'] == 'trip_end'):
                is_completed = True
                max_all = max_time
            else:
                max_all = 60 * 60

            travel_time = max_all - min_all

            if is_new and is_completed:
                #print(f"Nuovo e completo")
                travel_times_finished[h][name] = travel_time
            elif is_new and not is_completed:
                #print(f"Nuovo e non completo")
                travel_times_not_finished[h][name] = travel_time
            elif not is_new and is_completed:
                #print(f"Non nuovo e completo")
                for h_prev in list_prev_h:
                    try:
                        #print(f"{h_prev} provata per vedere se c'è il nome")
                        travel_time_prev = travel_times_not_finished[h_prev][name]
                        travel_times_finished[h_prev][name] = travel_time_prev + travel_time
                        del travel_times_not_finished[h_prev][name]
                        break
                    except:
                        print(f"Il nome non c'era")
                        continue
            elif not is_new and not is_completed:
                print(f"Non nuovo e non completo")
                for h_prev in list_prev_h:
                    try:
                        travel_time_prev = travel_times_not_finished[h_prev][name]
                        travel_times_not_finished[h_prev][name] = travel_time_prev + travel_time
                        break
                    except:
                        continue

    rows = [(outer_k, inner_k, value) for outer_k, inner_dict in travel_times_finished.items() for inner_k, value in inner_dict.items()]
    df = pd.DataFrame(rows, columns=['hour', 'veh_name', 'travel_time'])
    df.to_parquet(f"results/AreaVerde_TT/AreaVerde_TT_v14_seed_{s}.parquet")

In [ ]:
# To check if different seeds lead to the same distribution
import seaborn as sns
import matplotlib.pyplot as plt

h = 18
ts = 1
list_s = [i for i in range(ts)]

for s in list_s:
    tt_red = pd.read_parquet(f"results/AreaVerde_TT/AreaVerde_TT_v14_seed_{s}.parquet")
    tt_red = tt_red[tt_red['hour']==h]
    tt_hour = tt_red if s==list_s[0] else pd.concat([tt_hour, tt_red])
    tt_hour['seed'] = f'{s}'

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
sns.histplot(data=tt_hour, x='travel_time', hue='seed' )
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

s = 0
th = 24
list_h = [i for i in range(th)]

for s in list_s:
    tt_red = pd.read_parquet(f"results/AreaVerde_TT/AreaVerde_TT_v14_seed_{s}.parquet")
    tt_hour = tt_red if s==list_s[0] else pd.concat([tt_hour, tt_red])
    tt_hour['seed'] = f'{s}'

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
sns.histplot(data=tt_hour, x='travel_time', hue='hour', stat="percent")
plt.show()


In [ ]:
x_min = tt_hour['travel_time'].min()
x_max = tt_hour['travel_time'].max()

fig, axes = plt.subplots(5, 5, figsize=(20, 16))
axes = axes.flatten() 

for hour in list_h:
    hour_data = tt_hour[tt_hour['hour'] == hour]
    
    ax = axes[hour]
    sns.histplot(data=hour_data, x='travel_time', ax=ax, kde=True, bins=50, stat='percent')
    ax.axvline(x=hour_data['travel_time'].mean(), color='red')
    
    ax.set_xlim(x_min, x_max)
    
    ax.set_title(f'Ora {hour}')
    ax.set_xlabel('Tempo di viaggio')
    ax.set_ylabel('Frequenza')

for i in range(24, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
#Check travel time mean
th = 24
list_h = [i for i in range(th)]
ts = 1
list_s = [i for i in range(ts)]
for h in list_h:
    for s in list_s:
        tt_red = pd.read_parquet(f"results/AreaVerde_TT/AreaVerde_TT_v14_seed_{s}.parquet")
        tt_red = tt_red[tt_red['hour'] == h]
        tt_red['seed'] = s
        tt_red = (tt_red
                  [['hour', 'seed', 'travel_time']]
                  .groupby(['hour', 'seed'])
                  .mean()
                  .reset_index()
        )
        tt = tt_red if h==list_h[0] and s==list_s[0] else pd.concat([tt, tt_red])
      

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

fig, ax = plt.subplots(figsize=(12, 6))

# Define line styles based on unique seed values
linestyles = ['-', '--', ':', '-.', (0, (3, 1, 1, 1)), (0, (5, 1)), (0, (3, 5, 1, 5))]
unique_seeds = tt['seed'].unique()
seed_to_style = {seed: linestyles[i % len(linestyles)] for i, seed in enumerate(unique_seeds)}

# Use a colormap for hours (assuming hours are between 0-23)
cmap = plt.cm.twilight
norm = mcolors.Normalize(vmin=tt['hour'].min(), vmax=tt['hour'].max())

# Plot each line with color based on hour and style based on seed
for i, row in tt.iterrows():
    color = cmap(norm(row['hour']))
    style = seed_to_style[row['seed']]
    ax.vlines(x=row['travel_time']/60, ymin=0, ymax=10, linewidth=2, color=color, linestyles=style)

# Add a colorbar to show the hour mapping
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('Hour of Day')

# Add a custom legend for the seed styles
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], color='black', lw=1, linestyle=style, 
                         label=f'Seed {seed}') for seed, style in seed_to_style.items()]
ax.legend(handles=legend_elements, loc='best')

ax.set_xlabel('Travel Time [min]')
ax.set_title('Travel Times [min] by Hour and Seed')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots()

sns.histplot(data=tt, x='travel_time', hue='hour', palette='twilight_shifted', legend=False)

# Add a colorbar to show the hour mapping
sm = plt.cm.ScalarMappable(cmap='twilight_shifted', norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('Hour of Day')

fig.show()

In [ ]:
fig, ax = plt.subplots()

sns.histplot(data=tt, x='travel_time', hue='seed', palette='viridis', legend=False)

# Add a colorbar to show the hour mapping
sm = plt.cm.ScalarMappable(cmap='viridis', norm=mcolors.Normalize(vmin=tt['seed'].min(), vmax=tt['seed'].max())
)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('Seed')

fig.show()

## Stucked vehicles and stucking links

In [ ]:
from uxsimulator.analysis import congestion_metrics

th = 24
ts = 5
dp = "results"
nv = "UXsim_vehicles/AreaVerde_vehicles_with_return_v5"

sv_all = []
for s in range(ts):
    sv_tmp = []
    for h in range(th):
        sv = congestion_metrics.count_hourly_vehicles(
            type_veh="stuck", #"full_stuck"
            hour=h,
            seed=s,
            namefile_vehicles=nv,
            datapath=dp
        )
        sv_tmp.append(sv)
#        if sv > 0:
#            print(f"Seed {s} --- Hour {h} has {sv} stucked vehicles")
    sv_all.append(sv_tmp)


df_sv_all = pd.DataFrame({
    'seed': [i for i in range(len(sv_all)) for _ in range(len(sv_all[i]))],
    'hour': [j for i in range(len(sv_all)) for j in range(len(sv_all[i]))],
    'count_stucked': [sv_all[i][j] for i in range(len(sv_all)) for j in range(len(sv_all[i]))]
})
df_sv_all = df_sv_all[df_sv_all['count_stucked']>0]

df_sv_all['count_stucked'].describe()

In [ ]:
# from constants import CRS_PROJECTED
# import uxsimulator.analysis.utils
# import uxsimulator.analysis.congestion_metrics

# import matplotlib.pyplot as plt
# import contextily as ctx
# import pandas as pd
# from matplotlib.colors import Normalize

# th = 24
# seed = 0
# n = 5
# dp = "data/results"
# nv = "UXsim_vehicles/AreaVerde_vehicles_with_return_v4"

# for h in range (th):
#     if sv_all[seed][h] > 10:
#         # Compute
#         link_stats = uxsimulator.analysis.congestion_metrics.identify_hourly_stucking_links(
#             stuck_hour=h, stuck_seed=seed, namefile_vehicles=nv, datapath=dp, n_stuck=n
#         )

#         # Prepare plot
#         nodes = uxsimulator.analysis.utils.read_output(namefile="nodes_v6.csv", datapath=dp, namecols=['node_id', 'x', 'y', 'AV_position'])
#         links = uxsimulator.analysis.utils.read_output(namefile="edges_v6.csv", datapath=dp, namecols=['link_id', 'u','v', 'length', 'maxspeed', 'lanes', 'id_zone'])
#         links = (
#             links
#             .merge(nodes.rename(columns={'node_id': 'u', 'x': 'x_origin', 'y': 'y_origin'}), on='u')
#             .merge(nodes.rename(columns={'node_id': 'v', 'x': 'x_dest', 'y': 'y_dest'}), on='v')
#             .rename(columns={'link_id': 'link'})
#         )
#         stuck_links = pd.merge(link_stats, links, on='link')

#         # Plot
#         fig, ax = plt.subplots(figsize=(4,4))
#         norm = Normalize(vmin=stuck_links['stuck_count'].min(), vmax=stuck_links['stuck_count'].max())
#         stuck_links = stuck_links.sort_values('stuck_count')
#         for id, link in stuck_links.iterrows():
#             if link['stuck_count'] == 0:
#                 plt.plot([link['x_origin'], link['x_dest']], [link['y_origin'], link['y_dest']], color='grey')    
#             else:
#                 plt.plot([link['x_origin'], link['x_dest']], [link['y_origin'], link['y_dest']], color='red', linewidth=norm(link['stuck_count'])*5)
#         plt.title(f"Stucking links in hour {h} of seed {seed}")
#         ctx.add_basemap(ax, crs=CRS_PROJECTED, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.2)
#         plt.show()


In [ ]:
import uxsimulator.analysis.utils

df = uxsimulator.analysis.utils.read_output(namefile="AreaVerde_vehicles_with_return_v5_from_7_to_8_seed_0.csv", datapath="results/UXsim_vehicles",dtype={0: str}).rename({'name': 'vehicle_id'}, axis=1)
df_ok = df.loc[df['link']=='trip_end', 'vehicle_id'].values
df = df[~df['vehicle_id'].isin(df_ok)]

In [ ]:
stuck_vehicles = []
grouped = df.groupby('vehicle_id')
n=5

for vehicle_id, group in grouped:
    group = group.sort_values(by='t', ascending=False).reset_index(drop=True)
    if group.loc[0, 'link'] == "waiting_at_origin_node":
        stuck_vehicles.append(vehicle_id)
    elif len(group) == 1:
        stuck_vehicles.append(vehicle_id)
    else:
        t1 = group.loc[0, 't']
        t0 = group.loc[1, 't']
        if t1-t0 > n * 60:
            stuck_vehicles.append(vehicle_id)
stuck_vehicles